In [ ]:
import flopy
import os
import geopandas as gpd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import geopandas as gpd
import basic
import contextily as ctx
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import os
import matplotlib.pyplot as plt
import pathlib
import conda_scripts.plot_help as ph
from scipy.spatial.distance import pdist, squareform


import flopy.utils.binaryfile as bf

import warnings

In [ ]:
info, swr_info, sfr_info, riv_keys_info = basic.load_params('June2016')

datestart = info['start_date']

name = info['name']

out_folder = basic.out_folder('June2016')


print(datestart)
print(out_folder)

In [ ]:
import conda_scripts.arich_functions as af

In [ ]:
basic

In [ ]:
grid = gpd.read_file("GIS/grid.shp")


In [ ]:
m = basic.load_model()


In [ ]:
routing = gpd.read_file('GIS/nhd_hr_demo_sfr_routing.shp')

In [ ]:
sfr = gpd.read_file('GIS/nhd_hr_demo_sfr_cells.shp')
sfr = sfr.query("name=='Russian River'")

fig, ax  = basic.map_river()

sfr.plot(ax = ax, color = 'r',zorder = 100)

In [ ]:
# bottom = 765


In [ ]:
# kper = None

In [ ]:
# def load_params():
    
#     import json

#     with open("model_params.txt") as json_data_file:
#         data = json.load(json_data_file)
    
#     info = data['calibration']
#     swr = data['calibration']['parameters']['SWR']
#     sfr = data['calibration']['parameters']['SFR']
#     riv_keys = data['riv_keys']
    
#     return info, swr, sfr, riv_keys



info, swr_info,sfr_info, riv_keys = basic.load_params()
riv_keys

In [ ]:
swr_info

In [ ]:
IGEOTYPE = swr_info["IGEOTYPE"] 
IGCNDOP = swr_info["IGCNDOP" ]
GMANNING = swr_info["GMANNING" ]   
LEAKANCE = swr_info["LEAKANCE"]
getextd = swr_info["getextd"] 
bottom = swr_info['dam_swr_rno']
reach_min = swr_info["reach_main"]
IROUTETYPE = swr_info["IROUTETYPE"]
use_average_reach_len = swr_info["use_average_reach_len"]
use_thalweg = swr_info['use_thalweg']
kper = None if swr_info["kper"].lower()=='none' else swr_info["kper"]
SWR_processor = swr_info["SWR_processor"]
rain = swr_info['rain']
evap = swr_info['evap']

kper

In [ ]:
# Function to find adjacent
# elements in List
def findAdjacentElements(index, test_list):
    res = []
    for idx, ele in enumerate(test_list):
    # for idx, ele in zip(index, test_list):
  
        # Checking for all cases to append
        if idx == 0:
            res.append((1, test_list[idx + 1]-1))
        elif idx == len(test_list) - 1:
            res.append((1, test_list[idx - 1]-1))
        else:
            res.append((2, test_list[idx - 1]-1, test_list[idx + 1]-1))
    return dict(zip(index, res))

def find_route(sfr, bottom, n = 10, reach_min = 578, IROUTETYPE  = 4, use_average_reach_len = False ):
    if reach_min is None:
        reach_min = bottom-n
    print(f'\nselecting reaches:\t{bottom} to {reach_min}')
    
    sfr = sfr.query(f"rno<={bottom} & rno>={reach_min} ")
    
    print('dropping duplicates')
    sh = sfr.shape
    sfr = sfr.drop_duplicates(['k','i','j'])
    print(f'\nold length: {sh[0]}\nnew length: {sfr.shape[0]}\n')
    
    sfr = sfr.sort_values('rno')
    
    min_r = sfr.rno.min() - 1 # offset to get reach 1 
    sfr.loc[:,'rno_og'] = sfr.loc[:,'rno'].copy()
    sfr.loc[:,'rno'] = np.arange(1, sfr.shape[0]+1)
    sfr.loc[:,'outreach'] = np.arange(2, sfr.shape[0]+2) #sfr.loc[:,'rno'].shift(-1, fill_value=-999)
    # sfr.loc[:,'rno'] =  sfr.loc[:,'rno'] - min_r
    # sfr.loc[:,'outreach'] =  sfr.loc[:,'outreach'] - min_r
    sfr.loc[:,'ieqn'] = IROUTETYPE 
    sfr.loc[:,'irgnum'] = sfr.loc[:,'rno']
    sfr.loc[:,'krch'] = 1 # layer 1
    sfr.loc[:,'IRCH'] = sfr.loc[:,'i']+1
    sfr.loc[:,'JRCH'] = sfr.loc[:,'j']+1
    
    # reach location data
    #IREACH IEQN IRGNUM KRCH   IRCH JRCH   RLEN
       # 1    4     01    1     161  62  304.8
       # 2    4     02    1     162  62  304.8
       # 3    4     03    1     163  61  304.8
       # 4    4     04    1     164  61  304.8
       # 5    4     05    1     165  61  304.8
    reach_loc = sfr.loc[:,['rno','ieqn', 'irgnum', 'krch', 'IRCH', 'JRCH', 'rchlen' ]]
    
    if use_average_reach_len:
        warnings.warn('\nusing average reach lengths, not actual reach lengths\n'*3)
        reach_loc.loc[:,'rchlen'] = reach_loc.loc[:,'rchlen'].mean()
    
    rout = findAdjacentElements(sfr.loc[:,'rno'].values, sfr.loc[:,'outreach'].values, )
        
    return sfr, reach_loc, rout

def write_reach_connect(rout):
    with open('RR_2022/inputs/rout.txt','w') as r:
        for key, value in rout.items():
            v = '    '.join([str(v) for v in value])
            r.write(f"{key}   {v}\n")
            
def write_reach_loc_dat(sfr_filt):
    sfr_filt.to_csv('RR_2022/inputs/reach_loc.txt', sep = ' ', index = False, header = False)

def write_stress_period(sfr_filt):
    '''
      # DATASET 5 - STRESS PERIOD 1
    # ITMP IRDBND IRDRAI IRDEVP IRDLIN IRDGEO IRDSTR IRDSTG IPTFLG [IRDAUX]
     1     5    0     0       1     5      1     5      1      # 5  
    
    '''

    n = sfr_filt.shape[0]
    with open("RR_2022/inputs/stress_period_data.txt",'w') as sp:
        sp.write("# DATASET 5 - STRESS PERIOD\n")
        sp.write( "# ITMP IRDBND IRDRAI IRDEVP IRDLIN IRDGEO IRDSTR IRDSTG IPTFLG [IRDAUX]\n")
        sp.write(f"     1     {n}    0     0       1     {n}      1     {n}      1      # 5")
        
def write_dataset_6(sfr_filt):
        with open('RR_2022/inputs/dataset6.txt','w') as r:
            for i in sfr_filt.loc[:,'rno']:
                r.write(f"{i}  1\n")
    
def write_dataset_evap(sfr_filt,evap =0.000):
    sfr_filt.loc[:,'evap'] = evap
    # sfr_filt.loc[:,['rno','evap']]
    sfr_filt.loc[:,['rno','evap']].rename(columns = {'rno':'#rno'}).to_csv('RR_2022/inputs/evap.tab', sep = ' ', index = False)
    
def write_dataset_rain(sfr_filt,rain =0.000):
    sfr_filt.loc[:,'rain'] = rain
    # sfr_filt.loc[:,['rno','evap']]
    sfr_filt.loc[:,['rno','rain']].rename(columns = {'rno':'#rno'}).to_csv('RR_2022/inputs/rain.tab', sep = ' ', index = False)
    
def write_dataset_10(sfr_filt, shift = 0):
    # DATASET 10 - GEOMETRY ASSIGNMENT DATA
    sfr_filt.loc[:,'IGMODRCH'] = sfr_filt.loc[:,'rno']
    sfr_filt.loc[:,'IGEONUMR'] = sfr_filt.loc[:,'rno']
    sfr_filt.loc[:,'GZSHIFT'] = shift
    #IGMODRCH IGEONUMR GZSHIFT 
    sfr_filt.loc[:,['IGMODRCH','IGEONUMR','GZSHIFT' ]].rename(columns = {"IGMODRCH":"#IGMODRCH"}).to_csv('RR_2022/inputs/dataset_10.txt', sep = ' ', index = False)
    

    
def write_dataset_14a(sfr_filt, m, shift = 5):
    bot = m.dis.top
    sfr_filt.loc[:,'top'] = bot[sfr_filt.loc[:,'i'], sfr_filt.loc[:,'j']] + shift
    
    # print(sfr_filt.loc[:,'top'])
    sfr_filt.loc[:,['rno', 'top']].to_csv('RR_2022/inputs/dataset_14a.txt', sep = ' ', index = False)
        
        
def write_start_stage(sfr_filt, stage_shift_from_mod_top = .5, use_thalweg = True, kper = None):
    '''
    uses ending heads of last model run to create new starting heads
    '''    
    path = pathlib.Path(m.model_ws, 'Results', 'ISWRPSTG.dat')
    # path = os.path.join
    if path.exists() and (os.path.getsize(path)>1000):
        ISWRPSTG = pd.read_csv(path).rename(columns = lambda x: x.strip())
        ISWRPSTG = ISWRPSTG.set_index(['TOTIME','SWRDT','KPER','KSTP','KSWR'])   
        
        if kper is not None:
            filt = f"KPER=={kper}"
            print(f'selecting starting heads with {filt}')
            ISWRPSTG = ISWRPSTG.query(filt)
        
        endh= ISWRPSTG.tail().mean().dropna().to_frame('endheads')
    else:
        class obj:
            def __init__(self):
                self.shape = [0,0]
        
        endh = obj()
        
    
    if endh.shape[0] == sfr_filt.shape[0] and not use_thalweg:
        print('using ending stage values from previous run')
        endh.insert(0,'reach', value  = np.arange(endh.shape[0]) + 1 )
        # display(endh.head())
        endh.rename(columns = {'reach':'#rno'}).to_csv('RR_2022/inputs/start_stage.tab', sep = '\t', index = False)
        return endh
    else:
        print('using stream thalwegs')
        df = pd.read_csv(os.path.join('RR_2022/inputs/stream_thalwegs.txt'))
        df.loc[:,'thalwegs'] = df.loc[:,'thalweg']+stage_shift_from_mod_top
        print(f'using thalweg + {stage_shift_from_mod_top:.2f}ft for each reach starting stage')

        df.loc[:,['reach','thalweg']].rename(columns = {'reach':'#rno'}).to_csv('RR_2022/inputs/start_stage.tab', sep = '\t', index = False)
        return df.loc[:,['reach','thalweg']].rename(columns = {'reach':'#rno'})
        
def plot_swr(sfr_filt):
    fig,ax = plt.subplots()
    
    ax.plot(sfr_filt.rno, sfr_filt.top)
    
    roll = sfr_filt.set_index('rno').loc[:,'top'].rolling(20,min_periods = 0).min()
    # display(roll)
    ax.plot(roll.index, roll.values)
    
    
# reach_min = 578
    # bottom, n = 10, reach_min = 578, IROUTETYPE  = 4,    
sfr_filt, reach_loc, rout = find_route(sfr, 
                                       bottom, 
                                       n = None, 
                                       reach_min = reach_min,
                                       IROUTETYPE = IROUTETYPE, 
                                       use_average_reach_len=use_average_reach_len)

print(sfr_filt.shape)

write_reach_connect(rout)

write_reach_loc_dat(reach_loc)

write_stress_period(sfr_filt)

write_dataset_6(sfr_filt)

write_dataset_evap(sfr_filt,evap)
write_dataset_rain(sfr_filt,rain)

write_dataset_10(sfr_filt)


write_dataset_14a(sfr_filt, m)




    
plot_swr(sfr_filt)

# reach_loc

In [ ]:
endh = write_start_stage(sfr_filt, stage_shift_from_mod_top =.51, use_thalweg = use_thalweg, kper = 10)

ax = endh.set_index('reach').rename(columns = {'endheads':'starting stage'}).plot(title = 'starting stage', marker = '.', figsize = (10,4))
ax.grid(True)
plt.savefig(os.path.join(out_folder, 'star_stage'))

# use model top as starting heads

In [ ]:
def set_top_as_start():
    hdsobj = bf.HeadFile(os.path.join(m.model_ws, 'Results','RRbin.hds'))
    hds = hdsobj.get_data(kstpkper=hdsobj.get_kstpkper()[-1])
    hds.shape

    fig, axes = plt.subplots(1, 3, figsize=(20, 8.5), subplot_kw=dict(projection=ccrs.epsg(2226)))
    axes = axes.flat
    grid = m.modelgrid
    for i, hdslayer in enumerate([m.dis.top.array, m.dis.top.array,m.dis.top.array]):

        filename = os.path.join(m.model_ws, 'inputs', f"start_head_lay{i+1}.txt")
        np.savetxt(filename, hdslayer, fmt = '%.3f', delimiter = ',')

        mapview = flopy.plot.PlotMapView(m,ax = axes[i])
        linecollection = mapview.plot_grid(linewidth = .3)

        quadmesh = mapview.plot_array(hdslayer, vmax = 80,vmin  =20,cmap = 'gist_ncar_r')
        ctx.add_basemap(axes[i], crs = 2226)
        mapview.plot_ibound()
        # ax.set_title(f"layer {lay+1} elevation (feet)")
        # Hide X and Y axes tick marks
        axes[i].set_xticks([])
        axes[i].set_yticks([])
        basic.set_bounds(axes[i], 'mirabel')

        # hdslayer[hdslayer==-999] = np.nan
        # im = axes[i].imshow(hdslayer, vmin=0, vmax=75)
        axes[i].set_title("Layer {}".format(i + 1))
        ctr = axes[i].contour(hdslayer, colors="k", linewidths=0.5, vmin = 0, vmax = 75)

        # export head rasters
        # (GeoTiff export requires the rasterio package; for ascii grids, just change the extension to *.asc)
        flopy.export.utils.export_array(
            grid, "Output_heads/heads{}.tif".format(i + 1), hdslayer
        )

        # export head contours to a shapefile
        flopy.export.utils.export_array_contours(
            grid, "Output_heads/heads{}.shp".format(i + 1), hdslayer
        )

    # fig.delaxes(axes[-1])
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.03, 0.7])
    fig.colorbar(quadmesh, cax=cbar_ax, label="Head");


# set ending heads as starting heads for model...

In [ ]:
vmax = 60
vmin  =20

In [ ]:

def set_starting_heads(kper = None):
    hdsobj = bf.HeadFile(os.path.join(m.model_ws, 'Results','RRbin.hds'))
    if kper is None:
        hds = hdsobj.get_data(kstpkper=hdsobj.get_kstpkper()[-1])
    else:
        kx = hdsobj.get_kstpkper()[kper]
        print(f"using stress period: {kx[1]}, ts {kx[0]} as starting heads")
        hds = hdsobj.get_data(kstpkper=kx)
    hds.shape

    fig, axes = plt.subplots(1, 3, figsize=(20, 8.5), subplot_kw=dict(projection=ccrs.epsg(2226)))
    axes = axes.flat
    grid = m.modelgrid
    for i, hdslayer in enumerate(hds):

        # if i==0:
        #     warnings.warn('setting thalweg elevations as starting heads')
        #     hdslayer[min_elev.loc[:,'i'],min_elev.loc[:,'j']] = min_elev.loc[:,'thalweg']

        filename = os.path.join(m.model_ws, 'inputs', f"start_head_lay{i+1}.txt")
        np.savetxt(filename, hdslayer, fmt = '%.3f', delimiter = ',')

        mapview = flopy.plot.PlotMapView(m,ax = axes[i])
        linecollection = mapview.plot_grid(linewidth = .3)

        quadmesh = mapview.plot_array(hdslayer, vmax = vmax,vmin  =vmin, cmap = 'gist_ncar_r')
        ctx.add_basemap(axes[i], crs = 2226)
        # ax.set_title(f"layer {lay+1} elevation (feet)")
        # Hide X and Y axes tick marks
        axes[i].set_xticks([])
        axes[i].set_yticks([])
        basic.set_bounds(axes[i], 'mirabel')

        sfr_filt.plot(ax = axes[i], facecolor = "None")

        # hdslayer[hdslayer==-999] = np.nan
        # im = axes[i].imshow(hdslayer, vmin=0, vmax=75)
        axes[i].set_title("Layer {}".format(i + 1))
        # ctr = axes[i].contour(hdslayer, colors="k", linewidths=0.5, vmax = 80,vmin  =20,cmap = 'gist_ncar_r')

        # export head rasters
        # (GeoTiff export requires the rasterio package; for ascii grids, just change the extension to *.asc)
        flopy.export.utils.export_array(
            grid, "Output_heads/heads{}.tif".format(i + 1), hdslayer
        )

        # export head contours to a shapefile
        flopy.export.utils.export_array_contours(
            grid, "Output_heads/heads{}.shp".format(i + 1), hdslayer
        )

    # fig.delaxes(axes[-1])
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.03, 0.7])
    fig.colorbar(quadmesh, cax=cbar_ax, label="Head")

    
# set_starting_heads(20)

# change starting heads as starting heads for model...


## burning in the streambed thalweg

In [ ]:
def set_start_with_burn():
    
    swr = gpd.read_file("GIS/SWR_Reaches.shp")
    min_elev = pd.read_csv(os.path.join('RR_2022/inputs/stream_thalwegs.txt')).set_index('reach')
    min_elev = pd.merge(min_elev, swr.loc[:,['i','j','rno']], left_index = True, right_on = 'rno').drop(columns = 'rno')
    strt = m.bas6.strt.array

    fig, axes = plt.subplots(1, 3, figsize=(20, 8.5), subplot_kw=dict(projection=ccrs.epsg(2226)))
    axes = axes.flat
    grid = m.modelgrid
    for i, hdslayer in enumerate(strt):

        if i==0:
            warnings.warn('setting thalweg elevations as starting heads')
            hdslayer[min_elev.loc[:,'i'],min_elev.loc[:,'j']] = min_elev.loc[:,'thalweg']

        filename = os.path.join(m.model_ws, 'inputs', f"start_head_lay{i+1}.txt")
        np.savetxt(filename, hdslayer, fmt = '%.3f', delimiter = ',')

        mapview = flopy.plot.PlotMapView(m,ax = axes[i])
        linecollection = mapview.plot_grid(linewidth = .3)

        quadmesh = mapview.plot_array(hdslayer, vmax = 80,vmin  =20, cmap = 'gist_ncar_r')
        ctx.add_basemap(axes[i], crs = 2226)
        # ax.set_title(f"layer {lay+1} elevation (feet)")
        # Hide X and Y axes tick marks
        axes[i].set_xticks([])
        axes[i].set_yticks([])
        basic.set_bounds(axes[i], 'mirabel')

        sfr_filt.plot(ax = axes[i], facecolor = "None")

        # hdslayer[hdslayer==-999] = np.nan
        # im = axes[i].imshow(hdslayer, vmin=0, vmax=75)
        axes[i].set_title("Layer {}".format(i + 1))
        # ctr = axes[i].contour(hdslayer, colors="k", linewidths=0.5, vmax = 80,vmin  =20,cmap = 'gist_ncar_r')

        # export head rasters
        # (GeoTiff export requires the rasterio package; for ascii grids, just change the extension to *.asc)
        flopy.export.utils.export_array(
            grid, "Output_heads/heads{}.tif".format(i + 1), hdslayer
        )

        # export head contours to a shapefile
        flopy.export.utils.export_array_contours(
            grid, "Output_heads/heads{}.shp".format(i + 1), hdslayer
        )

    # fig.delaxes(axes[-1])
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.03, 0.7])
    fig.colorbar(quadmesh, cax=cbar_ax, label="Head");

# plot swr model cells

In [ ]:
def swr_map():
    fig, ax = basic.map_river()
    sfr_filt = gpd.read_file('GIS/SWR_reaches.shp')
    sfr_filt.plot(ax = ax, facecolor = 'w', edgecolor = 'y', lw = .1, zorder = 100)

    sfr_filt.loc[[sfr_filt.rno.idxmax()],:].geometry.centroid.plot(ax = ax, marker = '*', edgecolor = 'c', facecolor = 'c', lw = 1, zorder =101)
    sfr_filt.loc[[sfr_filt.rno.idxmin()],:].geometry.centroid.plot(ax = ax,marker = '*', edgecolor = 'orange', facecolor = 'orange', lw = 1, zorder =101)

    ax.text(sfr_filt.loc[[sfr_filt.rno.idxmax()],:].geometry.centroid.x, sfr_filt.loc[[sfr_filt.rno.idxmax()],:].geometry.centroid.y, 'Dam',zorder = 102)
    ax.text(sfr_filt.loc[[sfr_filt.rno.idxmin()],:].geometry.centroid.x, sfr_filt.loc[[sfr_filt.rno.idxmin()],:].geometry.centroid.y, 'Inflow',zorder = 102)

    return fig, ax

fig,ax = swr_map()

plt.savefig(os.path.join(out_folder, 'sfr_swr_map.png'), dpi = 250)

In [ ]:
xsect_shape = gpd.read_file(r"T:\arich\Russian_River\MirabelWohler_2022\GIS\GEO\RAS_xsections.shp")

In [ ]:
def find_(reach, str):
    x= next(line for line in reach if  line.strip().startswith(str))
    return x

def read_data(reach):
    lines = []
    for line in reach:
        if 'END' in line.strip():
            break
        else:
            lines.append([float(x.strip()) for x in line.split(',') if (len(x.strip())>0)])
    return lines

# read and import the cross section data exported from HEC-RAS.

In [ ]:
stream_network = {}
xsect = {}
cnt = 1
with open(r"T:\arich\Russian_River\MirabelWohler_2022\GIS\GEO\RAS.RASexport.sdf", 'r') as reach:
    # next(line for line in reach if  line.startswith("BEGINSTREAMNETWORK:"))
    x = find_(reach,'BEGINSTREAMNETWORK')
    print(x)
    # get next three lines into a list (empty string for nonexistent lines)
    # results = [next(reach, "").rstrip() for line in range(3)]
    # print(results) 
    find_(reach,'CENTERLINE:')
    stream_network = read_data(reach)

    find_(reach,'BEGIN CROSS-SECTIONS:')
    while reach:
        try:
            station = find_(reach, 'STATION').strip().split(':')[1]
            find_(reach, 'SURFACE LINE')
            section = read_data(reach)

            xsect[f"REACH_{cnt}"] = section
            cnt = cnt+1
        except:
            break

print(len(xsect.items()))


# convert cross sections to gdf, then find the nearest cross section to sfr_filt centroids

In [ ]:
dfall = pd.DataFrame()
cnt = 0
for items in xsect.items():
    df = pd.DataFrame(items[1], columns = ['x','y','z'])
    df.loc[:,'xsect'] = items[0]
    df.loc[:,'dist'] = squareform(pdist(df.loc[:,['x','y']]))[0,:]
    df.loc[:,'NGEOPTS'] = df.shape[0]
    dfall = pd.concat([dfall,df],ignore_index = True)
    cnt=+1

dfall = gpd.GeoDataFrame(dfall, geometry = gpd.points_from_xy(dfall.loc[:,'x'], dfall.loc[:,'y']), crs = 2226)

def joinish(sfr_filt, dfall):
    
    s_ = sfr_filt.copy()
    
    sfr_filt.loc[:,'gcent'] = sfr_filt.geometry.centroid
    
    sfr_filt =sfr_filt.set_geometry('gcent')
    
    sfr_filt = sfr_filt.sjoin_nearest(dfall.loc[:,['geometry','xsect', 'NGEOPTS']], how = 'left')
    
    sfr_filt = sfr_filt.set_geometry('geometry').drop(columns = 'gcent')
    
    return sfr_filt
    
sfr_filt_with_xsec = joinish(sfr_filt, dfall)



In [ ]:
fig, ax = basic.map_river()
sfr_filt_with_xsec.plot('xsect',ax = ax, zorder = 100)
dfall.plot('xsect', ax = ax, markersize = 1)


plt.savefig(os.path.join(out_folder, 'xsects_loc.png'), dpi = 250)

# export xsections


In [ ]:
folder= os.path.join(m.model_ws,'xsections')

if not os.path.exists(folder):
    os.mkdir(folder)

In [ ]:
sfr_filt_with_xsec.head()

In [ ]:
def smooth_thalweg(min_elev):
    min_elev = min_elev.loc[:,['THALWEG']].rename(columns = {'THALWEG':'THALWEG_Smoothed'})
    ind = min_elev.index

    for i in range(100):
        min_elev[min_elev.diff().shift(-1)>0] = np.nan
        min_elev = min_elev.dropna()
    min_elev = min_elev.reindex(ind)
    min_elev = min_elev.bfill()
    
    return min_elev

In [ ]:

def write_dataset_11a_with_geo(sfr_filt, 
                               min_elev = None,
                               IGEOTYPE = 3, 
                               IGCNDOP = 1, 
                               GMANNING = 0.025,   
                               LEAKANCE = .000001, 
                               getextd = 0.25, 
                               extra = 0.25):
    '''
    write geo section with actual cross section data
    
    '''
    # IGCNDOP = 0, Fixed conductance is specified for the geometry entry.
    # DATASET 11A - GEOMETRY DATA
# IGEONUM IGEOTYPE IGCNDOP GMANNING NGEOPTS GWIDTH GBELEV GSSLOPE    GCND      GLK GCNDLN GETEXTD
#      1    5         1      .25                                     9.2E-04                   0.25
    
    if min_elev is None:
        print('using observed thalwegs, then smoothing')
        min_elev ={}
        for xxx in sfr_filt.loc[:,['rno','NGEOPTS', 'xsect']].values:
            IGEONUM = xxx[0]
            NGEOPTS = xxx[1]
            xs = xxx[2]
            cur = dfall.query(f"xsect=='{xs}'")
            cur = cur.loc[:,['dist','z']]
            min_elev[IGEONUM] = cur.loc[:,'z'].min()
            
        min_elev = pd.DataFrame.from_dict(min_elev,orient='index',columns = ['THALWEG'])
        min_elev.index = min_elev.index.set_names('reach')
        
        mv = smooth_thalweg(min_elev)
        min_elev = min_elev.join(mv)
        # min_elev.loc[:,'THALWEG_Smoothed'] = min_elev.ewm(span = 20).mean()
    
    with open('RR_2022/inputs/dataset11a_with_geo.txt','w') as r:
        r.write("# DATASET 11A - GEOMETRY DATA\n")
        r.write("# IGEONUM IGEOTYPE IGCNDOP GMANNING NGEOPTS GWIDTH GBELEV GSSLOPE    LEAKANCE      GLK GCNDLN GETEXTD\n")
    
    with open('RR_2022/inputs/dataset11a_with_geo.txt','a', newline = '') as r:
        for xxx in sfr_filt.loc[:,['rno','NGEOPTS', 'xsect']].values:
            IGEONUM = xxx[0]
            NGEOPTS = xxx[1]
            xs = xxx[2]
            r.write(f"    {IGEONUM}\t{IGEOTYPE}\t{IGCNDOP}\t{GMANNING}\t{NGEOPTS}\t{LEAKANCE} \n")


            cur = dfall.query(f"xsect=='{xs}'")
            cur = cur.loc[:,['dist','z']]
            minz = cur.loc[:,'z'].min() - min_elev.at[IGEONUM, 'THALWEG_Smoothed']
            cur.loc[:,'z'] = cur.loc[:,'z'] -minz

            cur.to_csv(r, sep = '\t',header = False, index = False)
    
    
    if sfr_filt.filter(regex = 'THALWEG').columns is not None:
        sfr_filt = sfr_filt.drop(columns = sfr_filt.filter(regex = 'THALWEG').columns)

    
    sfr_filt = pd.merge(sfr_filt, min_elev, left_on = 'rno', right_index = True )
    sfr_filt = gpd.GeoDataFrame(sfr_filt, geometry = 'geometry', crs = 2226)
    
    min_elev.to_csv('RR_2022/inputs/stream_thalwegs.txt')
    min_elev.plot()
    # with open('RR_2022/inputs/stream_thalwegs.txt','w') as mint:
    #     mint.write('reach,THALWEG\n')
    #     [mint.write("{}, {}\n".format(item[0], item[1])) for item in min_elev.items()]
    #         # r.write('geo geo geo\n')
           
    return min_elev, sfr_filt

# min_elev, sfr_filt_with_xsec = 

min_elev, sfr_filt_with_xsec =  write_dataset_11a_with_geo(sfr_filt_with_xsec,
                           IGEOTYPE = IGEOTYPE, 
                           IGCNDOP = IGCNDOP, 
                           GMANNING =GMANNING,   
                           LEAKANCE = LEAKANCE, 
                           getextd = getextd, 
                           )

In [ ]:

sfr_filt_with_xsec.drop_duplicates('xsect').query(f"xsect=='REACH_418'").NGEOPTS.values[0]

In [ ]:
# import warnings
# def write_dataset_11a_with_geo_alt(sfr_filt, min_elev = None, 
#                                    IGEOTYPE = 3, IGCNDOP = 1, GMANNING = 0.025,   LEAKANCE = .000001, getextd = 0.25, extra = 0.25):
#     '''
#     write geo with constant cross section data, ie reusing cross sections
    
#     '''
#     # IGCNDOP = 0, Fixed conductance is specified for the geometry entry.
#     # DATASET 11A - GEOMETRY DATA
# # IGEONUM IGEOTYPE IGCNDOP GMANNING NGEOPTS GWIDTH GBELEV GSSLOPE    GCND      GLK GCNDLN GETEXTD
# #      1    5         1      .25                                     9.2E-04                   0.25

#     if min_elev is None:
#         print('using observed thalwegs, then smoothing')
#         min_elev ={}
#         for xxx in sfr_filt.loc[:,['rno','NGEOPTS', 'xsect']].values:
#             IGEONUM = xxx[0]
#             NGEOPTS = xxx[1]
#             xs = xxx[2]
#             cur = dfall.query(f"xsect=='{xs}'")
#             cur = cur.loc[:,['dist','z']]
#             min_elev[IGEONUM] = cur.loc[:,'z'].min()
            
#         min_elev = pd.DataFrame.from_dict(min_elev,orient='index',columns = ['THALWEG'])
#         min_elev.index = min_elev.index.set_names('reach')
#         min_elev.loc[:,'THALWEG_Smoothed'] = min_elev.ewm(span = 20).mean()
    
#     warnings.warn('\nsimple geo\n'*10)
    
#     with open('RR_2022/inputs/dataset11a_with_geo.txt','w') as r:
#         r.write("# DATASET 11A - GEOMETRY DATA\n")
#         r.write("# IGEONUM IGEOTYPE IGCNDOP GMANNING NGEOPTS GWIDTH GBELEV GSSLOPE    LEAKANCE      GLK GCNDLN GETEXTD\n")
    
#     with open('RR_2022/inputs/dataset11a_with_geo.txt','a', newline = '') as r:
#         for xxx in sfr_filt.loc[:,['rno','NGEOPTS', 'xsect']].values:
#             IGEONUM = xxx[0]
#             xs = 'REACH_418'
#             NGEOPTS = sfr_filt.drop_duplicates('xsect').query(f"xsect=='{xs}'").NGEOPTS.values[0]

            
#             # xs = 1
#             r.write(f"    {IGEONUM}\t{IGEOTYPE}\t{IGCNDOP}\t{GMANNING}\t{NGEOPTS}\t{LEAKANCE} \n")


#             cur = dfall.query(f"xsect=='{xs}'")
#             cur = cur.loc[:,['dist','z']]

#             minz = cur.loc[:,'z'].min()  - min_elev.at[IGEONUM, 'THALWEG_Smoothed']
#             # minz = cur.loc[:,'z'].min() - (39-IGEONUM*.1)

#             cur.loc[:,'z'] = cur.loc[:,'z'] -minz


#             cur.to_csv(r, sep = '\t',header = False, index = False)
            
#     assert not((min_elev.diff().thalweg>=0).any()), 'there are positive changes in elevation from reach to reach'
    
#     # with open('RR_2022/inputs/stream_thalwegs.txt','w') as mint:
#     #     mint.write('reach,thalweg\n')
#     #     [mint.write("{}, {}\n".format(item[0], item[1])) for item in min_elev.items()]
#     #         # r.write('geo geo geo\n')
    
#     if 'THALWEG' in sfr_filt.columns:
#         sfr_filt = sfr_filt.drop(columns = 'THALWEG')
#     sfr_filt = pd.merge(sfr_filt, min_elev, left_on = 'rno', right_index = True )
#     sfr_filt = gpd.GeoDataFrame(sfr_filt, geometry = 'geometry', crs = 2226)
           
#     return min_elev, sfr_filt
# min_elev, sfr_filt_with_xsec = write_dataset_11a_with_geo_alt(sfr_filt_with_xsec,
#                            IGEOTYPE = 3, 
#                            IGCNDOP = 0, 
#                            GMANNING = 0.35,   
#                            LEAKANCE = 0.0030, 
#                            getextd = 0.25, 
#                            )

# min_elev.add(3.0).to_csv(os.path.join('RR_2022', 'inputs', 'start_stage.tab'), sep = '\t',header = None)

In [ ]:
sfr_filt_with_xsec.to_file('GIS/SWR_Reaches.shp')
sfr_filt_with_xsec.to_file(r"T:\arich\Russian_River\MirabelWohler_2022\GIS\SWR_reaches.shp")

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Create figure
fig = go.Figure()

stepnames = {}
# Add traces, one for each slider step
for _,step in sfr_filt_with_xsec.drop_duplicates('xsect').iterrows():
    IGEONUM = step['rno']
    xs = step.loc['xsect']
    cur = dfall.query(f"xsect=='{xs}'")
    cur = cur.loc[:,['dist','z']]
    
    stepnames[IGEONUM] = f"reach: {IGEONUM} xs: {xs}"
    
    minz = cur.loc[:,'z'].min() - min_elev[IGEONUM]
    cur.loc[:,'z'] = cur.loc[:,'z'] -minz
    
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#00CED1", width=6),
            name="𝜈 = " + xs,
            x=cur.loc[:,'dist'],
            y=cur.loc[:,'z']))

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
# for i in range(len(fig.data)):
for i, stepper in enumerate(stepnames.keys()):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              # {"title": "Slider switched to step: " + str(i)}],  # layout attribute
                {"title": "Slider switched to step: " + stepnames[stepper]}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

In [ ]:
numdays = 3300

In [ ]:
cnt = 1
for _ in range(numdays+1):
   
    print(
    r"# ITMP IRDBND IRDRAI IRDEVP IRDLIN IRDGEO IRDSTR IRDSTG IPTFLG [IRDAUX]   #\
    1   -1    0      0       0     -1      2      0       1      # 5              #\
    open/close 'ref\nstruct.dat'\n")
    print(f"    open/close 'ref\dam_stage\day{cnt}.dat")
    
    cnt = cnt+1

In [ ]:
import basic
import numpy as np
import flopy
import os
def write_model_output_control(ml, num_days):
    head = 'HEAD PRINT FORMAT   0\n\
HEAD SAVE UNIT   336\n\
DRAWDOWN PRINT FORMAT   0\n\n'
    with open(os.path.join(ml.model_ws, 'RR.oc'), 'w') as oc:
        oc.write(head)
        for sp in np.arange(1, num_days+1):
            oc.write("period {:}  step 6  \nPRINT BUDGET\nSAVE HEAD\n".format(sp))


In [ ]:
ml = basic.load_model()

In [ ]:
ml.model_ws

In [ ]:
ml = basic.load_model(path = "temp\\SWR_long_2lays_med_SWRK")

In [ ]:
[h, hds] = basic.get_heads(ml, False)

In [ ]:
hds.get_kstpkper()

In [ ]:
ml.dis.write_file()

In [ ]:
ml.dis.botm[0].format.array_free_format

In [ ]:
oc = write_model_output_control(ml, 3286, )

In [ ]:
oc.write_file()

In [ ]:
o = ml.oc

In [ ]:
dis = ml.dis

In [ ]:
dis.nstp = 100

In [ ]:
dis.perlen = 86400

In [ ]:
dis

In [ ]:
path = ml.model_ws
verbose = False, 
# path = None
nam = 'RRMF.nam'
check = False
forgive = True
load_only = ['DIS', 'BAS6', 'UPW', 'NWT', 'WEL', 'GHB', 'OC', 'SFR', 'RCH', 'GAGE']
load_fast = True

In [ ]:
ml.dis.array_free_format

In [ ]:
ml.external_fnames

In [ ]:
ml = flopy.modflow.Modflow.load(nam,
                                load_only= load_only,
                                model_ws = path,
                                verbose = verbose,
                                forgive=forgive,
                                version = 'mfnwt', 
                                exe_name = f"{path}/MODFLOW-NWT_64.exe",
                                check = check)

In [ ]:
dis.write_file()

In [ ]:
o.write_file()

In [ ]:
import flopy

m = flopy.modflow.Modflow()

spd = {(0, 0): ['print head'],

  (0, 1): [],

  (0, 249): ['print head'],

  (0, 250): [],

  (0, 499): ['print head', 'save ibound'],

  (0, 500): [],

  (0, 749): ['print head', 'ddreference'],

  (0, 750): [],

  (0, 999): ['print head']}

oc = flopy.modflow.ModflowOc(m, stress_period_data=spd, cboufm='(20i5)')

oc.write_file()

In [ ]:
o.stress_period_data

In [ ]:
o